# Comparing Classification Methods

In this problem, we compare using three methods for classification: Ridge Regression, which we've seen over and over gain in this course, a soft-margin Support Vector Machine, which was the de-facto method for classifcation for a large chunk of machine learning's history, and LASSO (Least Absolute Shrinkage and Selection Operator) created by our colleagues over at Stanford. Although Robert Tibshirani (author of the paper that introduced LASSO) aknwoeldges and thanks Leo Breiman (very famous Berkeley professor that unforunately passed away in 2005) for sharing his garotte paper with him before publication. 

The problem we will try to tackle is a very simple binary classification problem in $R^2$. The training data has a variety of coordinates in $R^2$ some of which have been assigned to class $C_0$ and other to class $C_1$. Your goal for each classification method is the following: given a coordinate, determine whether this point belongs to $C_0$ or $C_1$. As we shall see even though all three methods attempt to solve the same problem, the way we forumalate the problem actually makes a difference, i.e. these methods will most likely not perform equally!

## Imports/Utils/Loading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import accuracy_score

from plot_boundary import plot_boundary

from numpy.polynomial.polynomial import polyval
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from sklearn.preprocessing import PolynomialFeatures


# load training data
train_data = np.load("data/train.npy")
X_train = train_data[:, 1:]
y_train = train_data[:, 0]

# load test data
test_data= np.load("data/test.npy")
X_test = test_data[:, 1:]
y_test = test_data[:, 0]

## Visualizing the data

To make the problem clearer, let's plot the training data and see what it looks like, lucklily being two-dimensional coordinates this is can be nicely done.

In the following plot the orange data points corresponds to $C_1$ (class = 1) and blue data points correspond to $C_0$ (class = 0). Something to reflect on: 
Is the data linearly separable? Why would we care?

In [ ]:
plt.scatter(X_train[y_train==0,0], X_train[y_train==0,1], alpha=0.4, s=5)
plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1], alpha=0.4, s=5)
plt.title("Training Data")
plt.show()

## Competitor N.1: Ridge Regression

Ridge regression is a regression technique that is quite similar to regular least squares linear regression: simply adding an $\ell_2$ **penalty** on the parameters $w$ to the objective function for linear regression yields the objective function for ridge regression.

Our goal is to solve the following problem:

$$\min_{w \in {R}^n} \|Xw - Y\|_2^2 + \lambda \|w\|_2^2,$$

where $\lambda$ is a hyperparameter and, $X$ is the training data and $Y$ the observations vector. In practice, we tune $\lambda$ until we find a model that generalizes well to the test data. There is no algorithm to find the best $\lambda$, usually what is done is we check various values and see which one gives the best results using the test data.

Ridge regression is an example of a **shrinkage method**: compared to least squares, it shrinks the parameter estimates in the hopes of **reducing variance, improving prediction accuracy, and aiding interpetation**. Intuitively, we see right away that $w$'s with large norms will probably not work even if our residual error is 0. 

### a) Fill out the following code block to run ridge regression to classify the data.

In [ ]:
fitted_model = None # your trained model (as trained by scipy)
y_pred_sign = None # the prediction of your trained model on the testing data

# convert the labels from 0 and 1 to -1 and 1
y_train_sign = np.array(y_train)
y_test_sign = np.array(y_test)
y_train_sign[y_train_sign == 0] = -1
y_test_sign[y_test_sign == 0] = -1

# for the regularization parameter lambda, try choosing values on different orders of magnitude.
llambda = 0.1

########## Your beautiful code starts here ##########

# TODO: train a fitted_model and run prediction to generate y_pred_sign 
# Hint lookup the sign function to get a prediction consistent with our class labels

y_pred_sign = None
fitted_model = None 

########## Your beautiful code ends here ##########

accuracy = accuracy_score(y_pred_sign, y_test_sign)
print("Test Accuracy: {}".format(accuracy))

plot_boundary(X_test, y_test, fitted_model)

## Competitor N.2: Least Absolute Shrinkage and Selection Operator (LASSO)

Lasso is somewhat similar to ridge regression, if you compare the formulation of the two problems you will notice that the only difference is in the penalty function. In ridge regression we shrink the coefficients since multipliying by $\lambda$ makes it unfavorable to pick $w$'s with large coefficients. LASSO, as the name suggests, not only shrinks the coefficients but it also "selects" some of them to be 0. In a hand wavey sense it attempts to understand which features (i.e. elememts) of $w$ don't really help us in the classification problem. 

Using LASSO, our goal is to solve the following problem:

$$\min_{w \in {R}^n} \|Xw - Y\|_2^2 + \lambda \|w\|_1,$$

where $\lambda$ is a hyperparameter and, $X$ is the training data and $Y$ the observations vector. In practice, we tune $\lambda$ until we find a model that generalizes well to the test data. There is no algorithm to find the best $\lambda$, usually what is done is we check various values and see which one gives the best results using the test data.

<!-- Fun fact: The problem was originally formulated as: $$\min_{w \in {R}^n} \|Xw - Y\|_2^2 : \|w\|_1 \leq \lambda$$ -->

### b) Fill out the following code block to run LASSO to classify the data.

In [ ]:
fitted_model_lasso = None # your trained model (as trained by scipy)
y_pred_sign_lasso = None # the prediction of your trained model on the testing data

# convert the labels from 0 and 1 to -1 and 1
y_train_sign = np.array(y_train)
y_test_sign = np.array(y_test)
y_train_sign[y_train_sign == 0] = -1
y_test_sign[y_test_sign == 0] = -1

# for the regularization parameter lambda, try choosing values of different orders of magnitude.
llambda = 0.1

########## Your beautiful code starts here ##########

# TODO: train a fitted_model and run prediction to generate y_pred_sign

y_pred_sign_lasso = None
fitted_model_lasso = None

########## Your beautiful code ends here ##########

accuracy = accuracy_score(y_pred_sign_lasso, y_test_sign)
print("Test Accuracy: {}".format(accuracy))

plot_boundary(X_test, y_test, fitted_model_lasso)

## Competitor N.3: Soft-Margin Support Vector Machine

In lecture and Q4 of HW8 you have been familiarized with what is known as the Hard-Margin SVM. In this problem we will be using the soft-margin SVM which is defined as follows:

$$\begin{align}
\min_{w \in \mathbb{R}^m ,\: b \in \mathbb{R},\: \zeta_i \in \mathbb{R}^n} \: \: & \frac{1}{2} \|w\|_2^2 + C \sum_{i=1}^n \zeta_i \\
\text{s.t. } & 1 - \zeta_i - y_i(x_i^\top w  - b) \leq 0 \\
& \zeta_i \geq 0
\end{align}$$

where $x_i$ is the $i$th data point, $y_i \in \{-1, 1\}$ is the label, and $C$ is a hyperparameter that controls how "soft" the margin is. 

Notice that we simply added some slack terms $\zeta_i$'s. Why did we do this? As you may know Hard-Margin SVM's require the data to be lienarly seprable otherwise you woulnd't have any feasible points. Soft-Margin SVM's allow us to find a decision boundary even if the data is not linearly separable, as it is the case in this exercise. By adding slack terms we are allowing some points to violate the margin, however, the more you violate the margin the "worse" you will do since you will be adding positive terms multiplied by some positive constant C to the objective, i.e. $ C \sum_{i=1}^n \zeta_i $ will drive objective value up as more points violate the margin.

If $C \to \infty$ then we get back our Hard-Margin SVM, however it would miserably fail in this case since the data is not linearly separable. Why is this? We see that if $C$ is very large then it becomes really expensive to violate the margin, since we are minimizing this is, the exact opposite of what we want.

Here is a good resource if you would like to understand Soft-Margin SVM's better: https://people.eecs.berkeley.edu/~jordan/courses/281B-spring04/lectures/lec6.pdf

Notice that this is more complicated than the other two regression problems, when picking a classification method this is also something you might want to take into account, how complicated your model is. 

### part c) Fill out the following code block to train a soft-margin svm and use it to classify the data.

In [ ]:
svc = None # your trained model
y_pred_sign = None # the prediction of your trained model on the testing data

########## Your beautiful code starts here ##########

# TODO: Write code to train an SVM, and generate prediction y_pred, choose different C values
c = 0.1
# The documentation for sklearn's SVM: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
# Note your classifier should be called "svc" as given in the beginning of the cell

y_pred_sign = None

########## Your beautiful code ends here ##########

accuracy = accuracy_score(y_pred_sign, y_test)
print("Test Accuracy: {}".format(accuracy)) 

plot_boundary(X_test, y_test, svc)